#  Feature Selection #

In [2]:
#Load libraries

#!pip install pyforest
from pyforest import *
import tsfresh


In [38]:
#LOAD DATA
df_train=pd.read_csv('C:/Users/Sergi Ch/Downloads/UBIQUM/@a SAMPLE PROJECTS/House Prices/House Prices/datasets/train.csv')
df_train_aux=df_train

### PREPARING THE DATA ###

In [39]:
#Data Cleaning

#missing data
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
#dealing with missing data
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max() #just checking that there's no missing data missing...

0

#### FEATURE SELECTION with tsfresh (only numeric)

In [43]:


Y=df_train['SalePrice']
X=df_train.loc[:, df_train.columns != 'SalePrice']

Xnum=X.select_dtypes(exclude=['object']) #exclude object variables


features=tsfresh.feature_selection.relevance.calculate_relevance_table(Xnum, Y, ml_task='regression', n_jobs=2, chunksize=None, test_for_binary_target_binary_feature='fisher', test_for_binary_target_real_feature='mann', test_for_real_target_binary_feature='mann', test_for_real_target_real_feature='kendall', fdr_level=0.05, hypotheses_independent=False)
#features
print('TSFRESH: Numeric Variables ' + str(len(Xnum.columns)))

TSFRESH: Numeric Variables 34


In [44]:
#Selecting features by tsfresh

features=features[features['relevant']==True]
#cols = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
X_tsfresh=df_train[features.index]

print('TSFRESH: Relevant Variables ' + str(len(X_tsfresh.columns)))

TSFRESH: Relevant Variables 25


### Univariate Statistical Tests (Chi-squared for classification)###

test for non-negative features to select 4 of the best features 

In [50]:

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
test=SelectKBest(score_func=chi2, k=4)
fit=test.fit(Xnum,Y)
# summarize scores
scores=fit.scores_ #choose attributes with the highest scores (plas, test, mass and age.)
features=fit.transform(Xnum)
features

array([[ 8450,   706,   854,     0],
       [ 9600,   978,     0,     0],
       [11250,   486,   866,     0],
       ...,
       [ 9042,   275,  1152,  2500],
       [ 9717,    49,     0,     0],
       [ 9937,   830,     0,     0]], dtype=int64)

### PCA Principal Component Analysis ###

In [52]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3) #extracting 3 principal components
fit=pca.fit(Xnum)
#info
fit.explained_variance_ratio_ #Explained Variance
fit.components_


array([[-1.40107935e-03, -5.92564918e-04,  9.99703598e-01,
         1.47948455e-05, -6.56967736e-07,  4.51001872e-05,
         2.98008527e-05,  9.82205508e-03,  1.79634465e-03,
        -1.03047730e-04,  1.15153520e-02,  1.16449561e-02,
         2.24581208e-03,  2.34664637e-05,  1.39142347e-02,
         8.24329534e-06,  1.14832507e-06,  7.00212224e-06,
         7.36489606e-07,  9.81291291e-06, -3.91741105e-07,
         3.10885354e-05,  1.75635949e-05,  1.16577859e-05,
         3.88286516e-03,  2.16116886e-03,  5.65671133e-04,
        -1.13075718e-04,  6.01290563e-05,  2.41916038e-04,
         3.13681988e-04,  1.89397779e-03,  3.33651299e-07,
        -1.89986420e-06],
       [ 1.19919607e-02, -3.15408496e-03, -2.26307287e-02,
         1.32246738e-03, -2.48749028e-04,  1.57473934e-02,
         1.06716659e-02,  2.36445179e-01, -9.73179981e-03,
         2.58122350e-01,  4.84835729e-01,  4.25362755e-01,
         1.94288095e-01,  3.19270023e-03,  6.22843550e-01,
         1.01282227e-04, -1.13

### Feature Importance ###
Random Forest and Extra Trees can be used to estimate the importance of features.


In [73]:
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier()
model.fit(Xnum, Y)
model.feature_importances_

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


array([0.05274496, 0.02619448, 0.05112038, 0.0339979 , 0.03057696,
       0.04920959, 0.04710178, 0.04407732, 0.0135622 , 0.05047915,
       0.04824255, 0.04879069, 0.02897279, 0.00231905, 0.0489909 ,
       0.02189962, 0.00588025, 0.01313004, 0.01539543, 0.02877517,
       0.00368293, 0.03628846, 0.02400248, 0.01978071, 0.04865171,
       0.03859701, 0.03843412, 0.01730178, 0.00337825, 0.01235179,
       0.000794  , 0.0054243 , 0.04889229, 0.04095897])

In [84]:
#CHECK RESULTS

results=pd.concat([pd.DataFrame(Xnum.columns), pd.DataFrame(model.feature_importances_) ], axis=1, ignore_index=True)
results=results.sort_values(by=[1], ascending=False)
results

,0,1
0,Id,0.052745
2,LotArea,0.051120
9,BsmtUnfSF,0.050479
5,YearBuilt,0.049210
14,GrLivArea,0.048991
32,MoSold,0.048892
11,1stFlrSF,0.048791
24,GarageArea,0.048652
10,TotalBsmtSF,0.048243
6,YearRemodAdd,0.047102
